In [8]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import yfinance as yf
import openai
from dotenv import load_dotenv, find_dotenv

In [9]:
# Numpy & Pandas tweaks for display
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

In [4]:
# load OPENAI_API_KEY from .env file
_ = load_dotenv(find_dotenv())

### Financial Parameter Analysis
A financial house will typically conduct a thorough Financial Parameter analysis of a target company to assess its financial health, stability and growth potential. These parameters are typically assesses from company's historical stock prices, its balance sheet, income statement, cash flow statement etc. 

In this section, we will calculate some sample parameters such as:
* Profitability Ratios:
    * Liquidity Ratios, such as Net Profit Margin and Return on Equity

<< TODO: complete this comment>>

In [52]:
def download_company_data(symbol: str):
    ticker = yf.Ticker(symbol)

    # fetch financial statements
    # income_stmt = ticker.income_stmt
    # balance_sheet = ticker.balance_sheet
    # cash_flow = ticker.cash_flow

    # download historical prices data (for last 5 years from today)
    from datetime import datetime, timedelta

    end_date = datetime.today()
    # start_date is 5 years previous to
    start_date = end_date - timedelta(days=5 * 365)
    market_data = ticker.history(
        start=start_date.strftime("%Y-%m-%d"), end=end_date.strftime("%Y-%m-%d")
    )

    return ticker, market_data

In [41]:
ticker, market_data = download_company_data("MSFT")

In [58]:
annual_dividends_per_share = ticker.dividends.resample("YE").sum()
close_prices_yearly = market_data["Close"].resample("YE").last()
dividend_yield = annual_dividends_per_share / close_prices_yearly
dividend_yield

Date
2019-12-31 00:00:00-05:00    0.003374
2020-12-31 00:00:00-05:00    0.009702
2021-12-31 00:00:00-05:00    0.007002
2022-12-31 00:00:00-05:00    0.010744
2023-12-31 00:00:00-05:00    0.007460
2024-12-31 00:00:00-05:00    0.005216
Freq: YE-DEC, dtype: float64

In [25]:
cash_flow.to_csv("~/Downloads/cash_flow_msft.csv", header=True, index=True)

In [46]:
def calculate_financial_ratios(symbol):
    ratios = {}

    # ticker, market_data, income_stmt, balance_sheet, cash_flow = download_company_data(
    #     symbol
    # )
    ticker, market_data = download_company_data(symbol)

    # calculate ratios

    # ---------- Profitability Ratios --------------
    # Net profit margin = Net Income / Total Revenue
    ratios["Net Profit Margin"] = (
        ticker.income_stmt.loc["Net Income"] / ticker.income_stmt.loc["Total Revenue"]
    )
    # Return on Equity (ROE) = Net Income / Total Stockhoder Equity
    ratios["ROE"] = (
        ticker.income_stmt.loc["Net Income"]
        / ticker.balance_sheet.loc["Stockholders Equity"]
    )
    # Return on Assets (ROA) = Net Income / Total Assets
    ratios["ROA"] = (
        ticker.income_stmt.loc["Net Income"] / ticker.balance_sheet.loc["Total Assets"]
    )

    # ---------- Liquidity Ratios --------------
    # Current Ratio = Current Assets / Current Liabilities
    ratios["Current Ratio"] = (
        ticker.balance_sheet.loc["Current Assets"]
        / ticker.balance_sheet.loc["Current Liabilities"]
    )
    # Quick Ratio = (Current Assets - Inventory) / Current Liabilities
    ratios["Quick Ratio"] = (
        ticker.balance_sheet.loc["Current Assets"]
        - ticker.balance_sheet.loc["Inventory"]
    ) / ticker.balance_sheet.loc["Current Liabilities"]

    # ---------- Leverage Ratios --------------
    # Debt to Equity = Total Liabilities / Stockholders Equity
    ratios["Debt To Equity"] = (
        ticker.balance_sheet.loc["Current Liabilities"]
        / ticker.balance_sheet.loc["Stockholders Equity"]
    )
    # Interest Coverage = EBIT / Interest Expense
    ratios["Interest Coverage"] = (
        ticker.income_stmt.loc["EBIT"] / ticker.income_stmt.loc["Interest Expense"]
    )

    # ---------- Efficiency Ratios --------------
    # Asset Turnover = Total Revenue / Total Assets
    ratios["Asset Turnover"] = (
        ticker.income_stmt.loc["Total Revenue"]
        / ticker.balance_sheet.loc["Total Assets"]
    )
    # Inventory Turnover = Cost of Revenue / Inventory
    ratios["Inventory Turnover"] = (
        ticker.income_stmt.loc["Cost Of Revenue"]
        / ticker.balance_sheet.loc["Inventory"]
    )

    # ---------- Valuation Metrics --------------
    # P/E Ratio = Market Price per Share / Earnings per Share (EPS)
    eps = ticker.financials.loc["Net Income"] / ticker.info["sharesOutstanding"]
    ratios["EPS"] = eps
    ratios["P/E Ratio"] = market_data["Close"].iloc[-1] / eps

    # ---------- Cash Flow Analysis --------------
    ratios["Operating Cash Flow"] = ticker.cash_flow.loc["Operating Cash Flow"]
    ratios["Free Cash Flow"] = (
        ticker.cash_flow.loc["Operating Cash Flow"]
        - ticker.cash_flow.loc["Capital Expenditure"]
    )

    # ---------- Divident Analysis --------------
    ratios["Dividend Yield"] = market_data

    return ratios


ratios = calculate_financial_ratios("MSFT")
ratios

{'Net Profit Margin': 2024-06-30     0.35956
 2023-06-30    0.341462
 2022-06-30    0.366863
 2021-06-30    0.364517
 dtype: object,
 'ROE': 2024-06-30    0.328281
 2023-06-30    0.350887
 2022-06-30    0.436755
 2021-06-30    0.431522
 dtype: object,
 'ROA': 2024-06-30    0.172086
 2023-06-30    0.175644
 2022-06-30     0.19937
 2021-06-30    0.183568
 dtype: object,
 'Current Ratio': 2024-06-30    1.274955
 2023-06-30    1.769167
 2022-06-30    1.784607
 2021-06-30    2.079994
 dtype: object,
 'Quick Ratio': 2024-06-30     1.26501
 2023-06-30    1.745163
 2022-06-30    1.745251
 2021-06-30    2.050261
 dtype: object,
 'Debt To Equity': 2024-06-30    0.466654
 2023-06-30    0.505031
 2022-06-30    0.570919
 2021-06-30    0.624398
 dtype: object,
 'Interest Coverage': 2024-06-30    37.724702
 2023-06-30    46.381606
 2022-06-30    41.579738
 2021-06-30    31.307758
 dtype: object,
 'Asset Turnover': 2024-06-30    0.478602
 2023-06-30    0.514387
 2022-06-30    0.543444
 2021-06-30    0

In [7]:
# Fetching hostorical market data (stocks info) for past 5 years
from datetime import datetime, timedelta

end_date = datetime.today()
# start_date is 5 years previous to
start_date = end_date - timedelta(days=5 * 365)
market_data = msft.history(
    start=start_date.strftime("%Y-%m-%d"), end=end_date.strftime("%Y-%m-%d")
)
market_data.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2019-09-18 00:00:00-04:00,131.199396,132.450640,130.406619,132.307373,23982100,0.0,0.0
2019-09-19 00:00:00-04:00,134.007527,135.984679,133.787846,134.742996,35772100,0.0,0.0
2019-09-20 00:00:00-04:00,134.685674,135.296969,132.049466,133.186096,39167300,0.0,0.0
2019-09-23 00:00:00-04:00,132.985542,133.367611,132.230980,132.899582,17139300,0.0,0.0
2019-09-24 00:00:00-04:00,134.064817,134.380019,130.740900,131.218475,29773200,0.0,0.0
